In [1]:
import numpy as np

import adaptoctree.morton as morton
import adaptoctree.tree as tree
import adaptoctree.plotting as plotting

# Test logic

In [16]:
N = int(1e3)
particles = plotting.make_moon(N)
# particles = np.random.rand(N, 3)

max_level = 16
max_num_particles = 76

In [17]:
max_bound, min_bound = morton.find_bounds(particles)
x0 = morton.find_center(max_bound, min_bound)
r0 = morton.find_radius(x0, max_bound, min_bound)

In [18]:
unbalanced = tree.build(particles, max_level, max_num_particles, 1)
tst = unbalanced
unbalanced = np.unique(unbalanced)
depth = max(morton.find_level(unbalanced))

In [19]:
len(unbalanced)

30

In [20]:
balanced = tree.balance(unbalanced, depth)

In [21]:
balanced = np.fromiter(balanced, np.int64, len(balanced))

In [22]:
len(balanced)

246

In [23]:
balanced_leaves = tree.assign_points_to_keys(particles, balanced, x0, r0)

In [24]:
%matplotlib

import itertools
import matplotlib.pyplot as plt

def plot_tree(octree, sources, octree_center, octree_radius):
    """

    Parameters:
    -----------
    octree : Octree
    """

    points = []

    fig1 = plt.figure()
    ax1 = fig1.add_subplot(111, projection='3d')

    unique = []

    for node in octree:
        level = morton.find_level(node)
        radius = octree_radius / (1 << level)

        center = morton.find_center_from_key(node, octree_center, octree_radius)

        r = [-radius, radius]

        for s, e in itertools.combinations(np.array(list(itertools.product(r, r, r))), 2):
            if np.sum(np.abs(s-e)) == r[1]-r[0]:
                ax1.plot3D(*zip(s+center, e+center), color="b")

    # Plot particle data
    ax1.scatter(sources[:, 0], sources[:, 1], sources[:, 2], c='g', s=10)
    plt.show()

Using matplotlib backend: Qt5Agg


In [27]:
plot_tree(unbalanced, particles, x0, r0)

In [28]:
plot_tree(balanced, particles, x0, r0)

In [12]:
# Test num particles constraint
_, counts = np.unique(balanced_leaves, return_counts=True)
assert np.all(counts <= max_num_particles)
assert sum(counts) == N

In [13]:
# Check for balancing condition
for i in balanced:
    for j in balanced:
        if (i != j) and morton.are_neighbours(i, j, x0, r0):
            diff = morton.find_level(i) - morton.find_level(j)
            assert diff <= 1

In [14]:
# Check for overlaps

for i, ki in enumerate(balanced):
    for j, kj in enumerate(balanced):
        if j != i:
            assert ki not in morton.find_ancestors(kj)            

# Benchmarking

In [23]:
N = int(1e5)
particles = plotting.make_moon(N)
# particles = np.random.rand(N, 3)

max_level = 16
max_num_particles = 5


max_bound, min_bound = morton.find_bounds(particles)
x0 = morton.find_center(max_bound, min_bound)
r0 = morton.find_radius(x0, max_bound, min_bound)

unbalanced = tree.build(particles, max_level=max_level, max_points=5)
unbalanced, counts = np.unique(unbalanced, return_counts=True)

len(unbalanced)

42770

In [24]:
np.all(counts <= 5)

True

In [25]:
%timeit tree.build(particles, max_level, max_num_particles)

34.3 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [26]:
%timeit tree.balance(unbalanced, depth)

17 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [27]:
tst = tree.balance(unbalanced, depth)

In [28]:
tst == set(unbalanced)

False